<a href="https://colab.research.google.com/github/Rob174/Astronomy/blob/Astronomy/AI/Mod%C3%A8le000001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fonctions de base

In [29]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/TIPE'
import os
from tensorflow.python.client import device_lib
print("Utilise le",str(device_lib.list_local_devices()[0])[15:18])
print()
print()
from google.colab import files
import tensorflow as tf
from tensorflow.python import debug as tf_debug

from keras.layers import Layer
import matplotlib.gridspec as gridspec
from keras.models import Sequential
from keras.layers import Dense, Dropout,concatenate,Subtract,Multiply,Average, Concatenate,Reshape, Add, BatchNormalization
from keras.layers import Reshape,Lambda
from keras.layers.core import Activation
from keras.optimizers import SGD, Adam
from keras import models
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.convolutional import Convolution2D, AveragePooling2D,MaxPooling2D
import keras.losses
from keras.layers.core import Flatten
from keras import backend as K
import numpy as np
import scipy
from PIL import Image
from keras.layers import Input
from keras.models import Model
import cv2
import matplotlib.pyplot as plt
images = ["Galaxies_resized/"+f for f in os.listdir("Galaxies_resized/")]
noises = []

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TIPE
Utilise le CPU




In [0]:
# Install latest Tensorflow build
# !pip install -q tf-nightly-2.0-preview
# from tensorflow import summary
# %load_ext notebookµ

In [0]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip

In [32]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.6/json/__init__.py", line 299, in load
    parse_constant=parse_constant, object_pairs_hook=object_pairs_hook, **kw)
  File "/usr/lib/python3.6/json/__init__.py", line 354, in loads
    return _default_decoder.decode(s)
  File "/usr/lib/python3.6/json/decoder.py", line 339, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/lib/python3.6/json/decoder.py", line 357, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


In [0]:
def next_batch(batch_size, images,tailleAttendue,formatArray):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    formatArray : format de sortie des données de l'image ; utiliser numpy
    """
    imageEntreeTensor = []
    imageSortieTensor = []
    while len(imageEntreeTensor) < batch_size:
        try:
            np.random.shuffle(images)#choix aléatoire de l'image
            image = cv2.imread(images[0])#Ouvre en rgb l'image nettoyée
            resizedImage = cv2.resize(image,(tailleAttendue,tailleAttendue))
            imageSortieTensor.append(np.array(resizedImage,dtype=formatArray))
            imageEntreeTensor.append(np.array(resizedImage,dtype=formatArray))
        except:
            print("Error in next_batch")
    imageEntreeTensor = np.array(imageEntreeTensor,formatArray)
    return [imageEntreeTensor,imageEntreeTensor]

In [0]:
def next_batch_bruit_voile(batch_size, images,tailleAttendue,formatArray,facteursVoile,bruitParam,plageVal=[0,255]):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    formatArray : format de sortie des données de l'image ; utiliser numpy
    facteurVoile : liste de valeur entre 0 et 1 contenant l'atténuation pour chaque couche de l'image
    bruitParam : liste avec dans l'ordre moyenne et écart type
    """
    assert plageVal[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]-bruitParam[1] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]+bruitParam[1] <= plageVal[1]
    
    imageEntreeTensor,imageSortieTensor = next_batch(batch_size,  images,tailleAttendue,formatArray)
    imageSortieTensorCopy = np.array(imageSortieTensor,dtype=np.float32)
    for image in range(imageSortieTensorCopy.shape[0]):
        for rgbIndex in range(3):
            imageSortieTensorCopy[image,:,:,rgbIndex] *= facteursVoile[rgbIndex]
    imageSortieTensorCopy = np.clip(imageSortieTensorCopy + np.random.normal(bruitParam[0],bruitParam[1],imageSortieTensorCopy.shape),plageVal[0],plageVal[1])
    return [imageEntreeTensor,np.array(imageSortieTensorCopy,dtype=formatArray)]

In [0]:
def next_batch_bruit_voile_2(batch_size, images,tailleAttendue,formatArray,facteursVoile,bruitParam,plageVal=[0,255]):#ATTENTION : pr tenter d'améliorer l'apprentissage, on augmente la taille minimale d'image prise
    """
    La versison 2 fait les  modification sélectives de couleurs après avoir ajouté le bruit
    formatArray : format de sortie des données de l'image ; utiliser numpy
    facteurVoile : liste de valeur entre 0 et 1 contenant l'atténuation pour chaque couche de l'image
    bruitParam : liste avec dans l'ordre moyenne et écart type
    """
    assert plageVal[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]-bruitParam[1] <= plageVal[1]
    assert plageVal[0] <= bruitParam[0]+bruitParam[1] <= plageVal[1]
    
    imageEntreeTensor,imageSortieTensor = next_batch(batch_size,  images,tailleAttendue,formatArray)
    imageSortieTensorCopy = np.array(imageSortieTensor,dtype=np.float32)
    for image in range(imageSortieTensorCopy.shape[0]):
        for rgbIndex in range(3):
            imageSortieTensorCopy[image,:,:,rgbIndex] += np.random.normal(bruitParam[0],bruitParam[1])
            imageSortieTensorCopy[image,:,:,rgbIndex] *= facteursVoile[rgbIndex]
    imageSortieTensorCopy /= np.max(imageSortieTensorCopy)
    imageSortieTensorCopy *= plageVal[1]
    imageSortieTensorCopy = np.clip(imageSortieTensorCopy,plageVal[0],plageVal[1])
    return [imageEntreeTensor,np.array(imageSortieTensorCopy,dtype=formatArray)]

In [0]:
def normalisation(arrayL,plageEntree=[0,255],plageSortie=[0,1]):
    assert plageEntree != plageSortie
    assert plageEntree[1]>0 and plageSortie[1] > 0
    formatArray = [array.dtype for array in arrayL]
    L = [np.array(array,dtype=np.float) for array in arrayL]
    for i in range(len(L)):
        L[i] = np.array((L[i]-plageEntree[0])/(plageEntree[1]-plageEntree[0])*(plageSortie[1]-plageSortie[0])+plageSortie[0],formatArray[i])
    return L

In [0]:
def LarrayFloatToUint(L):
    return [np.array(array,np.uint) for array in L]

In [0]:
def tauxApprentissage(epoch,ampl,tau,lim):
    taux = ampl*10**-((epoch)/tau)
    return taux if taux > lim else lim

In [0]:
def differenceAcceptee(epoch,ampl,tau,lim):
    taux = ampl*10**-((epoch)/tau)
    return taux if taux > lim else lim

In [0]:
def convertToUint(array):
    return np.array(normalisation(array,[0,1],[0,255]),dtype=np.uint8)

In [0]:
def convertToUintL(L):
    Lresult = []
    print("Entree : ",len(L))
    for i in range(len(L)):
        Lresult.append(np.array(normalisation(L[i],[0,1],[0,255]),dtype=np.uint8))
    print("Sortie : ",len(Lresult))
    return Lresult

In [0]:
index = np.arange(0,len(images))
np.random.shuffle(index)
trainingData = [images[i] for i in index[:int(0.6*len(images))]]
evalData = [images[i] for i in index[int(0.6*len(images))-1:]]

In [0]:
# source : https://stackoverflow.com/questions/46418373/how-to-resize-interpolate-a-tensor-in-keras
def interpolation(h,w,inputTensor):
    def resize_like(inputTensor,h,w):
        return tf.image.resize_nearest_neighbor(inputTensor, [h, w])

    return Lambda(resize_like, arguments={'h':h,'w':w})(inputTensor)

In [0]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
def SELU(x):
    return 1.0507*K.elu(x,alpha=1.67326)

get_custom_objects().update({'custom_activation': Activation(SELU)})

# A mettre pour le modèle : Activation(SELU)

In [0]:
class LRN2D(Layer):#Normalisation de réponse locale
    """
    This code is adapted from pylearn2.
    License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    """

    def __init__(self, alpha=1e-4, k=2, beta=0.75, n=5, **kwargs):
        if n % 2 == 0:
            raise NotImplementedError("LRN2D only works with odd n. n provided: " + str(n))
        super(LRN2D, self).__init__(**kwargs)
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n

    def get_output(self, train):
        X = self.get_input(train)
        b, ch, r, c = K.shape(X)
        half_n = self.n // 2
        input_sqr = K.square(X)
        extra_channels = K.zeros((b, ch + 2 * half_n, r, c))
        input_sqr = K.concatenate([extra_channels[:, :half_n, :, :],
                                   input_sqr,
                                   extra_channels[:, half_n + ch:, :, :]],
                                  axis=1)
        scale = self.k
        for i in range(self.n):
            scale += self.alpha * input_sqr[:, i:i + ch, :, :]
        scale = scale ** self.beta
        return X / scale

    def get_config(self):
        config = {"name": self.__class__.__name__,
                  "alpha": self.alpha,
                  "k": self.k,
                  "beta": self.beta,
                  "n": self.n}
        base_config = super(LRN2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [0]:
# cf https://stackoverflow.com/questions/43784921/how-to-display-custom-images-in-tensorboard-using-keras?noredirect=1#comment85726690_43784921
def make_image(tensor):
    """
    Convert an numpy representation image to Image protobuf.
    Copied from https://github.com/lanpa/tensorboard-pytorch/
    """
    from PIL import Image
    height, width = tensor.shape
    image = Image.fromarray(tensor).convert('RGB')
    import io
    output = io.BytesIO()
    image.save(output, format='PNG')
    image_string = output.getvalue()
    output.close()
    CHANNEL = 1
    return tf.Summary.Image(height=height,
                         width=width,
                         colorspace=CHANNEL,
                         encoded_image_string=image_string)

In [0]:
nb_couches = 3
def AI_gen_disc_or_disc_without_input(inpt,part='gen_disc'):
    print("Construction "+part)
    global Llayers
    global index_couches
    if part == 'gen_disc' or part == 'gen':
        with K.name_scope('Generateur_FFT'):
            def fft2d(x):
                x = K.cast(x,dtype=tf.complex64)
                for i in range(3):
                    print(x.get_shape().as_list())
                    fft_slice = Lambda(tf.fft2d, output_shape=(None,199,199,3))(x[:,:,:,i])
                    # d'après https://gist.github.com/akshaychawla/02849170e190fbd7fa9d431450e8d6ef
                    def assign(inpts):
                        return tf.assign(inpts[0],inpts[1])
                    x = Lambda(assign, output_shape=(None,199,199,3))([x[:,:,:,i],fft_slice])
                    # See https://stackoverflow.com/questions/34220532/how-to-assign-a-value-to-a-tensorflow-variable
                    print("Step")
                print(x)
                x = K.cast(x,dtype=tf.float32)
                return x
            couche = fft2d(inpt)#, output_shape=(None,199,199,3),name=str(index_couches)+'_'+'FFT2d')(inpt)
            index_couches += 1
            def magn_angle(x):
                magn_angle_tensor = tf.zeros(shape=(199,199,3,2))
                for i in range(3):
                    magnitude = tf.math.abs(x[:,:,i])
                    phase = tf.math.angle(x[:,:,i])
                    magn_angle_tensor = tf.assign(magn_angle_tensor[:,:,i,0],magnitude)
                    magn_angle_tensor = tf.assign(magn_angle_tensor[:,:,i,1],phase)
                return magn_angle_tensor
                    
#             couche = Lambda(magn_angle,name=str(index_couches)+'_'+'Magnitude_angle')(inpt)
            couche = Convolution2D(filters=500,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f500_k2')(couche)
            index_couches += 1
            couche = Dropout(name=str(index_couches)+'_'+'dropout',rate=0.25)(couche)
            index_couches += 1
            couche = LRN2D(name=str(index_couches)+'_'+'batchnorm',n=21,k=2,alpha=10**-4,beta=0.75)(couche)
            index_couches += 1
            couche = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(couche)
            index_couches += 1
#             Llayers.append([couche,str(index_couches)+'_'+'activation_SELU'])
            couche = Convolution2D(filters=33,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f33_k2')(couche)
            index_couches += 1
            couche = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(couche)
            index_couches += 1
            couche_FFT = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(couche)
            index_couches += 1
            def to_rgb(x):
                channel_ifft = tf.zeros(shape=x[:,:,:,:].as_list()[:-1])
                for i in raneg(3):
                    complex_val = x[:,:,i,0]*K.exp(tf.ones(shape=x[:,:,i,1].get_shape().as_list())*1j)
                    channel_ifft[:,:,i] = tf.ifft2d(complex_val)
                return channel_ifft
#             couche_FFT = Lambda(to_rgb,name=str(index_couches)+'_'+'fft_to_img')
            index_couches += 1
        with K.name_scope('Generateur_image'):
            couche = Convolution2D(filters=500,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f500_k2')(inpt)
            index_couches += 1
            couche = Dropout(name=str(index_couches)+'_'+'dropout',rate=0.25)(couche)
            index_couches += 1
            couche = LRN2D(name=str(index_couches)+'_'+'batchnorm',n=21,k=2,alpha=10**-4,beta=0.75)(couche)
            index_couches += 1
            couche = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(couche)
            index_couches += 1
#             Llayers.append([couche,str(index_couches)+'_'+'activation_SELU'])
            couche = Convolution2D(filters=100,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f100_k2')(couche)
            index_couches += 1
            couche = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(couche)
            index_couches += 1
            couche = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(couche)
            index_couches += 1
            couche = Concatenate(axis=-1,name=str(index_couches)+'_'+'Merge_img_fft')([couche,couche_FFT])
#             Llayers.append([couche,str(index_couches)+'_'+'activation_SELU'])
            couche = Convolution2D(filters=3,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f3_k2')(couche)
            index_couches += 1
            couche = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(couche)   
            index_couches += 1
            couche = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(couche)
            index_couches += 1
            couche = Dense(nb_couches,activation=None,name=str(index_couches)+'_'+'dense_n3')(couche)
            index_couches += 1
            couche = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(couche)   
            index_couches += 1
            couche = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(couche)
            index_couches += 1
            # couche = Add(name=str(index_couches)+'_'+'add')([couche,inpt])
            Llayers.append([couche,str(index_couches)+'_'+'activation_SELU'])
            # generator_ouput = couche
            if part == 'gen':
                return couche
    if part == 'gen_disc' or part == 'disc':
        with K.name_scope('Discriminateur'):
            with K.name_scope('Bruit'):
                if part == 'gen_disc':
                    bruit = Convolution2D(filters=1,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f1_k2')(couche)
                else:
                    bruit = Convolution2D(filters=1,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f1_k2')(inpt)
                index_couches += 1
                bruit = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(bruit)
                index_couches += 1
                bruit = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(bruit)
                index_couches += 1
                bruit = Convolution2D(filters=nb_couches,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f3_k2')(bruit)
                index_couches += 1
                bruit = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(bruit)
                index_couches += 1
                bruit = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(bruit)
                index_couches += 1
                if part == 'gen_disc':
                    bruit = Subtract(name=str(index_couches)+'_'+'subtract')([couche,bruit])
                else:
                    bruit = Subtract(name=str(index_couches)+'_'+'subtract')([inpt,bruit])
                index_couches += 1
#                 Llayers.append([bruit,str(index_couches)+'_'+'subtract'])
                bruit = MaxPooling2D(name=str(index_couches)+'_'+'maxpooling',pool_size=3,padding='VALID')(bruit)
                index_couches += 1
                bruit1 = Convolution2D(filters=1,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f1_k2')(bruit)
                index_couches += 1
                bruit1 = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(bruit)
                index_couches += 1
                bruit1 = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(bruit)
                index_couches += 1
                bruit1 = Convolution2D(filters=nb_couches,kernel_size=(2,2),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f_k2')(bruit)
                index_couches += 1
                bruit1 = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(bruit)
                index_couches += 1
                bruit1 = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(bruit)
                index_couches += 1
                bruit = Subtract(name=str(index_couches)+'_'+'subtract')([bruit,bruit1])
                index_couches += 1
#                 Llayers.append([bruit,str(index_couches)+'_'+'subtract'])
                bruit = MaxPooling2D(name=str(index_couches)+'_'+'maxpooling',pool_size=3,padding='VALID')(bruit)
                index_couches += 1
#                 Llayers.append([bruit,'disc_bruit_3'])
                
            with K.name_scope('Traitement_image'):
        
                def inception(prevShapes):
                    import time
                    global index_couches
                    inpt = Input(shape = prevShapes,name=str(index_couches)+'_'+'inception_input')
                    index_couches += 1
                    coucheT0 = Convolution2D(filters=100,kernel_size=(1,1),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f100_k1')(inpt)
                    index_couches += 1
                    coucheT1 = Convolution2D(filters=100,kernel_size=(1,1),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f100_k1')(inpt)
                    index_couches += 1
                    coucheT1 = Convolution2D(filters=100,kernel_size=(1,3),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f100_k1_3')(coucheT1)
                    index_couches += 1
                    coucheT1 = Convolution2D(filters=100,kernel_size=(3,1),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f100_k3_1')(coucheT1)
                    index_couches += 1

                    couche = Concatenate(axis=-1,name=str(index_couches)+'_'+'concatenate')([coucheT0,coucheT1])
                    index_couches += 1
                    couche = Convolution2D(filters=100,kernel_size=(1,1),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f100_k1')(couche)
                    index_couches += 1
                    couche = Add(name=str(index_couches)+'_'+'add')([couche,inpt])  
                    return Model(input=inpt,output=couche,name=str(index_couches)+'_'+'Inception_%f'%(time.time()))
                
                if part == 'gen_disc':
                    coucheAdaptation = Convolution2D(filters=100,kernel_size=(1,1),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f100_k1')(couche)
                else:
                    coucheAdaptation = Convolution2D(filters=100,kernel_size=(1,1),activation=None,strides=(1,1),padding='SAME',name=str(index_couches)+'_'+'Conv_f100_k1')(inpt)
                index_couches += 1
                inceptionModel = inception(coucheAdaptation.get_shape().as_list()[1:])
                index_couches += 1
                image = inceptionModel(coucheAdaptation)
                index_couches += 1
                image = LRN2D(name=str(index_couches)+'_'+'local_response_normalizer',n=21,k=2,alpha=10**-4,beta=0.75)(image)
                index_couches += 1
                image = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(image)
                index_couches += 1
                image = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(image)
                index_couches += 1
#                 Llayers.append([image,str(index_couches)+'_'+'activation_SELU'])
                index_couches += 1
                image = MaxPooling2D(name=str(index_couches)+'_'+'maxpooling',pool_size=3,padding='VALID')(image)
                index_couches += 1
                inceptionModel = inception(image.get_shape().as_list()[1:])
                index_couches += 1
                image = inceptionModel(image)
                index_couches += 1
                image = LRN2D(name=str(index_couches)+'_'+'local_response_normalizer',n=21,k=2,alpha=10**-4,beta=0.75)(image)
                index_couches += 1
                image = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(image)
                index_couches += 1
                image = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(image)
                index_couches += 1
#                 Llayers.append([image,str(index_couches)+'_'+'activation_SELU'])
                index_couches += 1
                image = MaxPooling2D(name=str(index_couches)+'_'+'maxpooling',pool_size=3,padding='VALID')(image)
                index_couches += 1
                inceptionModel = inception(image.get_shape().as_list()[1:])
                index_couches += 1
                image = inceptionModel(image)
                index_couches += 1
                image = LRN2D(name=str(index_couches)+'_'+'local_response_normalizer',n=21,k=2,alpha=10**-4,beta=0.75)(image)
                index_couches += 1
                image = BatchNormalization(name=str(index_couches)+'_'+'batchnorm')(image)
                index_couches += 1
                image = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(image)
                index_couches += 1
#                 Llayers.append([image,str(index_couches)+'_'+'activation_SELU'])
                index_couches += 1
        
            resultatAnalyse = Concatenate(axis=-1,name=str(index_couches)+'_'+'concatenate')([bruit,image])
            index_couches += 1
            resultatAnalyse = Flatten(name=str(index_couches)+'_'+'flatten')(resultatAnalyse)
            index_couches += 1
            resultatAnalyse = Dense(500,name=str(index_couches)+'_'+'dense_n500')(resultatAnalyse)
            index_couches += 1
            resultatAnalyse = Dropout(rate=0.25,name=str(index_couches)+'_'+'dropout')(resultatAnalyse)
            index_couches += 1
            resultatAnalyse = Activation(SELU,name=str(index_couches)+'_'+'activation_SELU')(resultatAnalyse)
            index_couches += 1
            probabilite = Dense(1,name=str(index_couches)+'_'+'dense_n1')(resultatAnalyse)
            index_couches += 1
            probabilite = Activation('sigmoid',name=str(index_couches)+'_'+'activation_sigmoid')(probabilite)
        return probabilite

In [0]:
def AI():
    inpt = Input(shape = (199, 199, nb_couches),name=str(0)+'_'+'Image')
    print("ok")
    probabilite_gen_disc = AI_gen_disc_or_disc_without_input(inpt,part='gen_disc')
    probabilite_disc = AI_gen_disc_or_disc_without_input(inpt,part='disc')
    generated_image = AI_gen_disc_or_disc_without_input(inpt,part='gen')
    return Model(input=inpt,output=probabilite_gen_disc),Model(input=inpt,output=probabilite_disc), Model(input=inpt,output=generated_image)

In [0]:
def mean_gen_disc_top_27(disc,gen,gen_disc,input_image,desired_image,indexModele):
    path_model = "Models/Modele"+str(indexModele)+"_"
    partial_path_gen_backup = path_model + 'gen' + '_'
    partial_path_gen_disc_backup = path_model + 'gen_disc' + '_'
    partial_path_disc_backup = path_model + 'disc' + '_'

    Lloss = []
    Lloss_gen = []
    Lloss_gen_disc = []
    Lloss_disc = []
    
    try:
        gen.save_weights(partial_path_gen_backup + 'orig.h5')
    except:
        gen.save_weights(partial_path_gen_backup + 'orig.h5')
    try:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    except:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    try:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    except:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')

    for i in range(3):
        gen_backup = partial_path_gen_backup + str(i) + '.h5'
        print("Training "+gen_backup)
        try:
            gen.load_weights(partial_path_gen_backup + 'orig.h5')
        except:
            gen.load_weights(partial_path_gen_backup + 'orig.h5')
        loss = gen.train_on_batch(input_image,desired_image)
        try:
            gen.save_weights(gen_backup)
        except:
            gen.save_weights(gen_backup)
        Lloss.append([gen_backup,loss])
    try:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    except:
        gen_disc.save_weights(partial_path_gen_disc_backup + 'orig.h5')
    try:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    except:
        disc.save_weights(partial_path_disc_backup + 'orig.h5')
    print("Lloss : ",Lloss)
    desired_probability = [1]*input_image.shape[0] # Batch size
    Lloss_gen_disc_tmp = []
    for index_gen_disc,path_loss_backup in enumerate(Lloss):
        try:
            gen.load_weights(path_loss_backup[0])
        except:
            gen.load_weights(path_loss_backup[0])
        for i in range(3):
            gen_disc_backup = partial_path_gen_disc_backup + str(i) + '.h5'
            print("Training "+gen_disc_backup)
            try:
                gen_disc.load_weights(partial_path_gen_disc_backup + 'orig.h5')
            except:
                gen_disc.load_weights(partial_path_gen_disc_backup + 'orig.h5')
            loss = gen_disc.train_on_batch(input_image,desired_probability)
            try:
                gen_disc.save_weights(gen_disc_backup)
            except:
                gen_disc.save_weights(gen_disc_backup)
            Lloss_gen_disc_tmp.append(path_loss_backup + [gen_disc_backup,loss])
    Lloss = Lloss_gen_disc_tmp
    print("Lloss : ",Lloss)

    desired_probability = [1]*(input_image.shape[0]//2)+[0]*(input_image.shape[0]//2)
    Lloss_disc_tmp = []
    for index_model,item in enumerate(Lloss):
        for i in range(3):
            disc_backup = partial_path_disc_backup + str(i) + '.h5'
            print("Training "+disc_backup)
            try:
                gen_disc.load_weights(Lloss[index_model][-2])
            except:
                gen_disc.load_weights(Lloss[index_model][-2])
            loss = disc.train_on_batch(input_image,desired_probability)
            try:
                disc.save_weights(disc_backup)
            except:
                disc.save_weights(disc_backup)
            Lloss_disc_tmp.append(Lloss[index_model]+[disc_backup,loss])


    Lloss = [Lloss_disc_tmp[i] + [(Lloss_disc_tmp[i][1]+Lloss_disc_tmp[i][3]+Lloss_disc_tmp[i][5])/3] for i in range(len(Lloss_disc_tmp))]
    print("Lloss : ",Lloss)
    Lloss.sort(key=lambda elem:elem[-1])
    print("Lloss sorted : ",Lloss)
    try:
        gen_disc.load_weights(Lloss[0][2])
        disc.load_weights(Lloss[0][-3])
    except:
        gen_disc.load_weights(Lloss[0][2])
        disc.load_weights(Lloss[0][-3])
    print("Best model back")
    return Lloss[0]

In [0]:
def sauvegardeModele(entree_pure,entree_deterioree,model,iteration_entrainement,summary_writer,batch_size=7):
    global Llayers
    for canal_image in range(3):
        summary_image = tf.Summary(value=[tf.Summary.Value(tag="input_pure_canal_"+str(canal_image), 
                                image=make_image(entree_pure[0,:,:,canal_image]*255))])
        summary_writer.add_summary(summary_image,iteration_entrainement)
        summary_image = tf.Summary(value=[tf.Summary.Value(tag="input_deterioree_canal_"+str(canal_image), 
                                image=make_image(entree_deterioree[0,:,:,canal_image]*255))])
        summary_writer.add_summary(summary_image,iteration_entrainement)
        
    for p,entree in enumerate([entree_pure,entree_deterioree]):
        layer_outputs,layer_names = [Llayers[i][0] for i in range(len(Llayers))],[Llayers[i][1] for i in range(len(Llayers))]
        print("layer_outputs : ",layer_outputs)
        model_calcul_image = Model(input=model.input,outputs=[model.output]+layer_outputs)
        sorties_couches = model_calcul_image.predict(entree, batch_size=batch_size)[1:]
        for index_couche,sortie_couche in enumerate(sorties_couches):
            layer_name = layer_names[index_couche]
            dim_sortie = sortie_couche.shape
            if len(dim_sortie) == 4:
                for canal_image in range(dim_sortie[-1]):
                    tag = layer_name
                    tag += 'pure' if p == 0 else 'deterioree'
                    tag += "_canal_" + str(canal_image)
                    summary_image = tf.Summary(value=[tf.Summary.Value(tag=tag, 
                                            image=make_image(sortie_couche[0,:,:,canal_image]*255))])
                    summary_writer.add_summary(summary_image,iteration_entrainement)
    return None

In [0]:
from google.colab import output
def beep():
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
    return None

In [0]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
continuer = None
def block_or_continue():
    beep()
    print("Would you resume training ? True or False ?")
    return bool(input())

In [0]:
def backup(image,imageBruitee,gen_disc,disc,index,summary_writer):
    x = np.concatenate((image,imageBruitee))
    sauvegardeModele(image,imageBruitee,gen_disc,index,summary_writer)
    p_gen_disc = gen_disc.predict(x)[0]
    p_disc = disc.predict(x)[0]

    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité gen-disc img 0 (orig pure) pure : ", 
                                        simple_value=p_gen_disc) ])
    summary_writer.add_summary(summary_loss,index)
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité disc img 0 (orig pure) pure : ", 
                                        simple_value=p_disc) ])
    summary_writer.add_summary(summary_loss,index)
    x_proba = np.concatenate((imageBruitee,imageBruitee))
    p_gen_disc = gen_disc.predict(x_proba)[0]
    p_disc = disc.predict(x_proba)[0]
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité gen-disc img 0 (orig détériorée) pure : ", 
                                        simple_value=p_gen_disc) ])
    summary_writer.add_summary(summary_loss,index)
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité disc img 0 (orig détériorée) pure : ", 
                                        simple_value=p_disc) ])
    summary_writer.add_summary(summary_loss,index)

In [0]:
def custom_loss(y_true,y_pred):
    return K.mean(K.square(y_pred - y_true),axis=-1)+K.max(K.square(y_pred - y_true),axis=-1)

In [0]:
def to_fft(img_tensor):
    L = []
    print(img_tensor.shape[0],img_tensor.get_shape().as_list())
    for index_image in range(img_tensor.get_shape().as_list()[0]):
        L.append(to_fft_single(img_tensor[index_image,:,:,:]))
    return np.stack(L,axis=0)

def to_img(fft_cplx):
    return to_uint8_array(np.fft.ifft2(fft_cplx))
def to_fft_single(img):
    fft_cplx = np.fft.fft2(img)
    fft = 20*np.log(np.abs(fft_cplx)+10**-5*np.ones(img.shape))
    fft = fft + np.abs(np.min(fft))*np.ones(fft.shape)
    fft = fft / (np.max(fft)*np.ones(fft.shape))*(255*np.ones(fft.shape))
    fft_uint8 = to_uint8_array(fft)
    print(np.max(fft_uint8),np.min(fft_uint8),fft_uint8.shape,fft_uint8.dtype)
    return fft_cplx

def to_uint8_array(array):
    return array.astype(np.uint8)
def next_batch_bruit_voile_fft():
    image,imageBruitee = next_batch_bruit_voile_2(7,images,199,np.float32,[1,1,1],[50,50])
    

In [56]:
index_couches = 0
Llayers = []
def train_gen_disc(images,eval_data,indexModele):
    gen_disc,disc,gen = AI()
    gen_disc_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    disc_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    gen_optim = SGD(lr=0.0005, momentum=0.9, nesterov=True)
    disc.compile(loss='binary_crossentropy', optimizer=disc_optim)
    gen_disc.compile(loss='binary_crossentropy', optimizer=gen_disc_optim)
    gen.compile(loss=custom_loss, optimizer=gen_optim)
    print("Layers : ",Llayers)
    print(gen_disc.summary())
    print(disc.summary())
    print(Llayers)
    Lloss = []
    iterations = 50
    index = 0
    import datetime
    chaine_date = datetime.datetime.today().strftime('%Y-%m-%d_%Hh%Mmin%Ss') #cf http://strftime.org/ et https://www.science-emergence.com/Articles/Obtenir-la-date-daujourdhui-au-format-YYYY-MM-DD-avec-python/
    summary_writer = tf.summary.FileWriter(logdir='./logs/Modele'+str(indexModele)+'_gen_disc_'+chaine_date,graph=tf.get_default_graph())
    while index < iterations:
        image,imageBruitee = next_batch_bruit_voile_2(7,images,199,np.float32,[1,1,1],[50,50])
        image,imageBruitee = to_fft_single(image),to_fft_single(imageBruitee)
        print("Max-min : ",np.max(image),np.min(image))
        current_learning_rate = tauxApprentissage(index,10**-4,10,10**-7)
        K.set_value(disc.optimizer.lr, current_learning_rate)
        K.set_value(gen_disc.optimizer.lr, current_learning_rate)
        current_learning_rate = tauxApprentissage(index,1,15,10**-7)
        K.set_value(gen.optimizer.lr, current_learning_rate)
        x = np.concatenate((image,imageBruitee))
        loss = mean_gen_disc_top_27(disc,gen,gen_disc,np.concatenate((image,imageBruitee)),np.concatenate((image,image)),indexModele)
        Lloss.append(loss)
        print("Iteration %i : gen, erreur : %f, gen_disc, erreur : %f ; disc, erreur : %f ; erreur moyenne : %f"%(index,loss[1],loss[3],loss[-2],loss[-1]))
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur gen : MSE", 
                                             simple_value=loss[1]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur gen-disc : binary-crossentropy", 
                                             simple_value=loss[3]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur disc : binary-crossentropy", 
                                             simple_value=loss[-2]) ])
        summary_writer.add_summary(summary_loss,index)
        summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Erreur moyenne : binary-crossentropy", 
                                             simple_value=loss[-1]) ])
        summary_writer.add_summary(summary_loss,index)
        
        if index % 5 == 0:
            backup(image,imageBruitee,gen_disc,disc,index,summary_writer)
        index += 1
    summary_writer.close()
Lloss = train_gen_disc(images,evalData,9)
# See https://stackoverflow.com/questions/48242585/tf-gradients-wont-work-with-tf-assign-but-works-with

ok
Construction gen_disc
[None, 199, 199, 3]


ValueError: ignored